<a href="https://colab.research.google.com/github/alirezaght/tesseract-train-font/blob/main/tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install tesseract

In [2]:
!apt install tesseract-ocr libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 5 newly installed, 0 to remove and 37 not upgraded.
Need to get 7,550 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libleptonica-dev amd64 1.75.3-3 [1,308 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtesseract-dev amd64 4.00~git2288-10f4998a-2 [1,447 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:5 http://archive.ubuntu.com/ubuntu

# Clone tesstrain

In [3]:
!git clone https://github.com/tesseract-ocr/tesstrain

Cloning into 'tesstrain'...
remote: Enumerating objects: 819, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 819 (delta 21), reused 18 (delta 9), pack-reused 775
Receiving objects: 100% (819/819), 13.37 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (463/463), done.


# Prepare data

In [13]:
!cp /content/drive/MyDrive/tesseract.rar /content/
!unrar x /content/tesseract.rar
!mv bnk-ground-truth tesstrain/data/bnk-ground-truth
%cd /content/tesstrain

Streaming output truncated to the last 5000 lines.
Extracting  ali/dca59a44-4e5c-4745-abab-9f828e99410d.tif                  85%  OK 
Extracting  ali/dcaae19c-c60b-4a5a-8d86-a0844295b075.gt.txt               85%  OK 
Extracting  ali/dcaae19c-c60b-4a5a-8d86-a0844295b075.tif                  85%  OK 
Extracting  ali/dcb559cb-4c10-4eb7-92b0-3db4e9f1cb40.gt.txt               85%  OK 
Extracting  ali/dcb559cb-4c10-4eb7-92b0-3db4e9f1cb40.tif                  85%  OK 
Extracting  ali/dcbd3338-b2d4-49cc-94a0-b824cbe4d1c0.gt.txt               85%  OK 
Extracting  ali/dcbd3338-b2d4-49cc-94a0-b824cbe4d1c0.tif                  85%  OK 
Extracting  ali/dcc6f7bd-6156-4a7b-b073-461a20a7e730.gt.txt               85%  OK 
Extracting  ali/dcc6f7bd-6156-4a7b-b073-461a20a7e730.tif                  85%  OK 
Extracting  ali/dcccdbeb-59d5-4811-9ae0-c44ddb771f14.gt.txt               85%  OK 
Extracting  ali/dcccdbeb-59d5

# Train

In [ ]:
!make training MODEL_NAME=bnk

Streaming output truncated to the last 5000 lines.
+ tesseract data/bnk-ground-truth/2fda484e-59f0-4066-85af-099b9904d662.tif data/bnk-ground-truth/2fda484e-59f0-4066-85af-099b9904d662 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.0.0-beta.1 with Leptonica
Page 1
Warning. Invalid resolution 0 dpi. Using 70 instead.
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/bnk-ground-truth/2fdb83ba-5b13-4d64-82e8-571bbb1c80b7.tif" -t "data/bnk-ground-truth/2fdb83ba-5b13-4d64-82e8-571bbb1c80b7.gt.txt" > "data/bnk-ground-truth/2fdb83ba-5b13-4d64-82e8-571bbb1c80b7.box"
+ tesseract data/bnk-ground-truth/2fdb83ba-5b13-4d64-82e8-571bbb1c80b7.tif data/bnk-ground-truth/2fdb83ba-5b13-4d64-82e8-571bbb1c80b7 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.0.0-beta.1 with Leptonica
Page 1
Warning. Invalid resolution 1 dpi. Using 70 instead.
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/bnk-ground-truth/2fe6c9ce-873f-46ef-be9b-3eb900afe4ac.tif" -t "data/bnk-ground